In [0]:
%python
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.datasets import load_iris
from time import time
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv")
X = df.drop('Outcome',axis=1)
y = df['Outcome']

# Load dataset
#data = load_iris()
#X = data.data
#y = data.target
scaler = StandardScaler()
X = scaler.fit_transform(X) 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

param_grid = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,),(64,), (128,), (64, 64), (128, 64), (128, 128)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05,0.001,0.01,0.1],
    'learning_rate': ['constant','adaptive'],
}
results = []
for hidden_layer_sizes, solver, activation, learning_rate, alpha in product(
    param_grid['hidden_layer_sizes'],
    param_grid['solver'],
    param_grid['activation'],
    param_grid['learning_rate'],
    param_grid['alpha']
):
    print(f"Tuning parameters: hidden_layer_sizes={hidden_layer_sizes}, solver={solver}, learning_rate={learning_rate}, activation={activation}, alpha={alpha}")
    start_time = time()
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        max_iter=500,
        early_stopping=True,
        validation_fraction=0.1,
        random_state=42
    )
    mlp.fit(X_train, y_train)
    end_time = time()
   
    # Evaluate on training set
    train_preds = mlp.predict(X_train)
    train_f1 = f1_score(y_train, train_preds, average='macro')
    train_accuracy = accuracy_score(y_train, train_preds)
    train_precision = precision_score(y_train, train_preds, average='macro', zero_division=0)
    train_recall = recall_score(y_train, train_preds, average='macro')
    
    # Evaluate on validation set
    valid_preds = mlp.predict(X_test)
    valid_f1 = f1_score(y_test, valid_preds, average='macro')
    valid_accuracy = accuracy_score(y_test, valid_preds)
    valid_precision = precision_score(y_test, valid_preds, average='macro', zero_division=0)
    valid_recall = recall_score(y_test, valid_preds, average='macro')

    results.append({
        'hidden_layer_sizes': hidden_layer_sizes,
        'solver': solver,
        'activation': activation,
        'train_f1': train_f1,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'valid_f1': valid_f1,
        'valid_accuracy': valid_accuracy,
        'valid_precision': valid_precision,
        'valid_recall': valid_recall,
        'training_time': end_time - start_time
    })

grid_search = GridSearchCV(
    estimator=MLPClassifier(max_iter=100, early_stopping=True, validation_fraction=0.1, random_state=42),
    param_grid=param_grid, cv=3, n_jobs=-1, verbose=2
)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_mlp = grid_search.best_estimator_
best_mlp.fit(X_train, y_train)
y_predictions = best_mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_predictions)
accuracy

Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=constant, activation=tanh, alpha=0.0001
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=constant, activation=tanh, alpha=0.05
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=constant, activation=tanh, alpha=0.001
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=constant, activation=tanh, alpha=0.01
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=constant, activation=tanh, alpha=0.1
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=adaptive, activation=tanh, alpha=0.0001
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=adaptive, activation=tanh, alpha=0.05
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, learning_rate=adaptive, activation=tanh, alpha=0.001
Tuning parameters: hidden_layer_sizes=(50, 50, 50), solver=sgd, lea

/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, 

/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: T


[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=adaptive, solver=sgd; total time=   0.4s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=adaptive, solver=adam; total time=   0.2s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=adaptive, solver=adam; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 128), learning_rate=adaptive, solver=adam; tota

...


[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=sgd; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=sgd; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=lbfgs; total time=   0.2s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=lbfgs; total time=   0.2s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=constant, solver=lbfgs; total time=   0.2

/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: T


[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(64,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(64,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(64,), learning_rate=constant, solver=sgd; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(64,), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(64,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(64,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(64,), learning_rate=constant, solver=lbfgs; total time=   0.2s
[CV] EN

/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/databricks/python/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: T

0.658008658008658

In [0]:
df.shape
df.head(100)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
95,6,144,72,27,228,33.9,0.255,40,0
96,2,92,62,28,0,31.6,0.130,24,0
97,1,71,48,18,76,20.4,0.323,22,0
98,6,93,50,30,64,28.7,0.356,23,0


In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predictions)

array([[107,  44],
       [ 35,  45]])

In [0]:
    results_df = pd.DataFrame(results)
    display(results_df.sort_values(by='valid_f1', ascending=False))

hidden_layer_sizes,solver,activation,train_f1,train_accuracy,train_precision,train_recall,valid_f1,valid_accuracy,valid_precision,valid_recall,training_time
List(100),lbfgs,relu,0.7362610161367004,0.7672253258845437,0.7459921798631476,0.7301255867829055,0.6857142857142857,0.7272727272727273,0.6975791772843102,0.6797185430463577,0.3181283473968506
List(100),lbfgs,relu,0.7362610161367004,0.7672253258845437,0.7459921798631476,0.7301255867829055,0.6857142857142857,0.7272727272727273,0.6975791772843102,0.6797185430463577,0.2503702640533447
List(100),lbfgs,relu,0.7362610161367004,0.7672253258845437,0.7459921798631476,0.7301255867829055,0.6857142857142857,0.7272727272727273,0.6975791772843102,0.6797185430463577,0.3251938819885254
List(100),lbfgs,relu,0.7362610161367004,0.7672253258845437,0.7459921798631476,0.7301255867829055,0.6857142857142857,0.7272727272727273,0.6975791772843102,0.6797185430463577,0.17380976676940918
"List(50, 50, 50)",lbfgs,relu,0.7447006636203259,0.7746741154562383,0.7547409579667643,0.7383100652319697,0.6595283509623722,0.7056277056277056,0.6712121212121211,0.6543460264900662,0.49202775955200195
"List(50, 50, 50)",lbfgs,relu,0.7447006636203259,0.7746741154562383,0.7547409579667643,0.7383100652319697,0.6595283509623722,0.7056277056277056,0.6712121212121211,0.6543460264900662,0.46242570877075195
"List(50, 50, 50)",lbfgs,relu,0.7447006636203259,0.7746741154562383,0.7547409579667643,0.7383100652319697,0.6595283509623722,0.7056277056277056,0.6712121212121211,0.6543460264900662,0.417621374130249
"List(50, 50, 50)",lbfgs,relu,0.7447006636203259,0.7746741154562383,0.7547409579667643,0.7383100652319697,0.6595283509623722,0.7056277056277056,0.6712121212121211,0.6543460264900662,0.548201322555542
"List(50, 100, 50)",lbfgs,logistic,0.7484358127421712,0.7802607076350093,0.7631341638555826,0.7401542400780345,0.6543133802816903,0.7056277056277056,0.671168161862951,0.6484685430463577,1.2666685581207275
"List(50, 100, 50)",lbfgs,logistic,0.7484358127421712,0.7802607076350093,0.7631341638555826,0.7401542400780345,0.6543133802816903,0.7056277056277056,0.671168161862951,0.6484685430463577,1.4172544479370117
